In [4]:
# Import relevant libraries
import pandas as pd

In [1]:
import json
with open("../data/levski.json", encoding='utf-8') as f:
    levski = json.load(f)

In [6]:
# Convert to pandas dataframe
df = pd.DataFrame(levski)
df

,opponent_name,levski_goals,opponent_goals,result,coach,hosted,scorers,date,tournament
0,ЦСКА (София),1,0,win,Ясен Петров,False,[Гара Дембеле],01 август 2010 година,"""А"" Футболна група"
1,Локомотив (София),3,1,win,Ясен Петров,True,"[Гара Дембеле, Гара Дембеле, Гара Дембеле]",09 август 2010 година,"""А"" Футболна група"
2,Черноморец Бургас (Бургас),1,2,loss,Ясен Петров,False,[Александър Александров],15 август 2010 година,"""А"" Футболна група"
3,Калиакра (Каварна),3,0,win,Ясен Петров,True,"[Александър Александров, Исмаил Иса]",22 август 2010 година,"""А"" Футболна група"
4,Видима-Раковски (Севлиево),2,1,win,Ясен Петров,False,"[Гара Дембеле, Дарко Тасевски]",30 август 2010 година,"""А"" Футболна група"
...,...,...,...,...,...,...,...,...,...
453,Берое (Стара Загора),1,1,tie,Станимир Стоилов,False,[Билал Бари],12 декември 2021 година,Първа професионална лига - първи етап
454,ЦСКА 1948 (София),2,0,win,Станимир Стоилов,False,"[Димитър Костадинов, Велтон Фелипе]",26 февруари 2022 година,Първа професионална лига - първи етап
455,Марек 1915 (Дупница),2,0,win,Станимир Стоилов,False,"[Димитър Костадинов, Димитър Костадинов]",22 септември 2021 година,Купа на България
456,Септември (Симитли),7,0,win,Станимир Стоилов,True,"[Марин Петков, Билал Бари, Билал Бари, Георги ...",26 октомври 2021 година,Купа на България


In [7]:
# Check for nulls
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   opponent_name   458 non-null    object
 1   levski_goals    458 non-null    int64 
 2   opponent_goals  458 non-null    int64 
 3   result          458 non-null    object
 4   coach           458 non-null    object
 5   hosted          458 non-null    bool  
 6   scorers         352 non-null    object
 7   date            458 non-null    object
 8   tournament      458 non-null    object
dtypes: bool(1), int64(2), object(6)
memory usage: 29.2+ KB


In [34]:
# OK. Scorers list can be null when no player scored.
# Translate months to digits, then convert to datetimes
months = {'януари' : '01','февруари' : '02','март' :'03','април': '04','май':'05','юни':'06','юли':'07','август':'08','септември':'09','октомври':'10','ноември':'11','декември':'12'}

df['date_dt'] = df['date'].replace(months,regex=True) # loop + str.replace faster?
df['date_dt'] = pd.to_datetime(df['date_dt'], format='%d %m %Y година',dayfirst=True)
df

,opponent_name,levski_goals,opponent_goals,result,coach,hosted,scorers,date,tournament,date_nums,date_dt
0,ЦСКА (София),1,0,win,Ясен Петров,False,[Гара Дембеле],01 август 2010 година,"""А"" Футболна група",01 8 2010 година,2010-08-01
1,Локомотив (София),3,1,win,Ясен Петров,True,"[Гара Дембеле, Гара Дембеле, Гара Дембеле]",09 август 2010 година,"""А"" Футболна група",09 8 2010 година,2010-08-09
2,Черноморец Бургас (Бургас),1,2,loss,Ясен Петров,False,[Александър Александров],15 август 2010 година,"""А"" Футболна група",15 8 2010 година,2010-08-15
3,Калиакра (Каварна),3,0,win,Ясен Петров,True,"[Александър Александров, Исмаил Иса]",22 август 2010 година,"""А"" Футболна група",22 8 2010 година,2010-08-22
4,Видима-Раковски (Севлиево),2,1,win,Ясен Петров,False,"[Гара Дембеле, Дарко Тасевски]",30 август 2010 година,"""А"" Футболна група",30 8 2010 година,2010-08-30
...,...,...,...,...,...,...,...,...,...,...,...
453,Берое (Стара Загора),1,1,tie,Станимир Стоилов,False,[Билал Бари],12 декември 2021 година,Първа професионална лига - първи етап,12 12 2021 година,2021-12-12
454,ЦСКА 1948 (София),2,0,win,Станимир Стоилов,False,"[Димитър Костадинов, Велтон Фелипе]",26 февруари 2022 година,Първа професионална лига - първи етап,26 2 2022 година,2022-02-26
455,Марек 1915 (Дупница),2,0,win,Станимир Стоилов,False,"[Димитър Костадинов, Димитър Костадинов]",22 септември 2021 година,Купа на България,22 9 2021 година,2021-09-22
456,Септември (Симитли),7,0,win,Станимир Стоилов,True,"[Марин Петков, Билал Бари, Билал Бари, Георги ...",26 октомври 2021 година,Купа на България,26 10 2021 година,2021-10-26
